<a href="https://colab.research.google.com/github/Vixuz144/Simulacion-1/blob/main/Guia_tercer_parcial_Figols.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from random import random, gauss
import numpy as np
from numba import jit

#1.-

El precio de una acción sigue un movimiento Browniano geométrico $P_t = p_0 e^{\frac{\mu - \sigma^2}{2} t + \sigma W_t}$ donde $W$, es un movimiento Browniano, con $\mu = 0.002$ $\sigma = 0.01$ y $P_0 = 10$. Sobre esta acción se define una opción europea con precio de ejercicio de 11 en un tiempo de 50.

A) Estime por simulación, el valor esperado de la ganancia (por acción).

B) Use la técnica de variables antitéticas para reducir la varianza. ¿En qué porcentaje se redujo?

#2

a) Estime por Simulacion el valor de la integral $∫_{0}^{2} e^{-x^2} dx$

b) Use la técnica devariables antitéticas para reducir la varianza. ¿En que porcentaje se redujo la varianza?

c) Use la técnica de variables de control con la variable $x^2$ como variable de control para reducir la varianza. ¿En que porcentaje se redujo la varianza?

In [ ]:
@jit(nopython = True)
def f2_anti(nr = 10**6):
  l, la = [], []
  for i in range(nr):
    x = 2*random()
    f = 2*np.exp(-x**2)
    g = 2*np.exp(-(2-x)**2)
    l.append(f)
    la.append((f+g)/2)
  return l, la

In [ ]:
l, la = f2_anti()

m, ma, v, va = np.mean(l), np.mean(la), np.var(l, ddof = 1), np.var(la, ddof = 1)
print(m, ma)
print(100 * (v-va)/v)

0.8829799279584771 0.8820531715030685
97.79998028434916


In [ ]:
@jit(nopython = True)
def f2_cont(nr = 10**6):
  l, lx, lc = [], [], []
  for i in range(nr):
    x = 2*random()
    l.append(2*np.exp(-x**2))
    lx.append(x**2)
  m = np.array(lx).mean()
  c = -np.cov(l,lx)[0,1]/np.cov(l,lx)[1,1]
  for i in range(nr):
    lc.append(l[i] + c*(lx[i]-m))
  return l, lc

In [ ]:
l, lc = f2_cont()
m, mc, v, vc = np.mean(l), np.mean(lc), np.var(l, ddof = 1), np.var(lc, ddof = 1)
print(m, mc)
print(100 * (v-vc)/v)

0.8812415892010932 0.8812415892010809
83.96955494088874


#3

La variable aleatoris $X$ sigue una distribución exponencial con parámetro 3.

A) Estime por simulación el valor esperado de $\sqrt X$.

B) Use la técnica de variables antitéticas para reducir la varianza, ¿En qué porcentaje se redujo ésta?

C) Use a la variable $X$ como variable de control para reducir la varianza, ¿En qué porcentaje se redujo ésta? Verifiquelo analiticamente.

D) Use la técnica de muestreo de importancia para reducir la varianza; use una función de densidad de una normal con media μ y desviación estándar σ. Determine estos parámetros para obtener la máxima reducción porcentual de la varianza. ¿En qué porcentaje se redujo ésta?

In [ ]:
@jit(nopython = True)
def f3_anti(nr = 10**6):
  l, la = [], []
  for i in range(nr):
    r = random()
    x = -np.log(r)/3
    a = -np.log(1-r)/3
    l.append(np.sqrt(x))
    la.append((np.sqrt(x) + np.sqrt(a))/2)
  return l, la

In [ ]:
l, la = f3_anti()
m, ma, v, va = np.mean(l), np.mean(la), np.var(l, ddof = 1), np.var(la, ddof = 1)
print(m, ma)
print(100 * (v-va)/v)

0.5121989858873999 0.5116706646855057
97.35324676998931


In [ ]:
@jit(nopython = True)
def f3_cont(nr = 10**6):
  l, lx, lc = [], [], []
  for i in range(nr):
    r = random()
    x = -np.log(r)/3
    l.append(np.sqrt(x))
    lx.append(x)
  m = np.array(lx).mean()
  c = -np.cov(l,lx)[0,1]/np.cov(l,lx)[1,1]
  for i in range(nr):
    lc.append(l[i] + c*(lx[i]-m))
  return l, lc

In [ ]:
l, lc = f3_cont()
m, mc, v, vc = np.mean(l), np.mean(lc), np.var(l, ddof = 1), np.var(lc, ddof = 1)
print(m, mc)
print(100 * (v-vc)/v)

0.5118571413201244 0.5118571413201141
91.50155583206205


Nota, no se puede


In [ ]:
@jit(nopython = True)
def f3_impo(m, s, nr = 10**6):
  l = []
  for i in range(nr):
    r = gauss(m,s)
    if r >= 0:
      x = np.sqrt(r)
      f = 3*np.exp(-3*r)
      h = np.exp(-(r-m)**2/(2*s**2))/(s*np.sqrt(2*np.pi))
      l.append(x*f/h)
    else:
      l.append(0)



  return l

In [ ]:
for m in np.arange(10,50,2):
  for s in np.arange(0.01,0.05, 0.01):
    l = f3_impo(m,s, 10**6)
    mi, vi = np.mean(l), np.var(l, ddof = 1)
    redvar =  round(100 * (v-vi)/v,4)
    if np.allclose(mc,mi, 10**-4) and 60 < redvar < 100:
      print('mu', round(m,2), 'sigma', round(s,2), 'media', round(mi,5), 'redvar', redvar, '\n')

#4

El precio de una acción sigue un movimiento Browniano geométrico $P_t = P_0 e^{\frac{\mu - \sigma^2}{2} t + \sigma W_t}$, donde $W$, es un movimiento Browniano, con $\mu = 0.002$, $\sigma = 0.01$ y $P_0= 5$.

A) Estime por simulación, el valor esperado del número de diss que tienen que transcurrir para que el precio de la acción sea superior a 10.4.

B) Reduzca la varianza usando la técnica de variables antitéticas, ¿En qué porcentaje se redujo la varianza?

C) Use variables de control para reducir la varianza, ¿En qué porcentaje se redujo la varianza?

In [ ]:
@jit(nopython = True)
def f4(m, s, p0, pa, nr = 10**6):
  l = []
  for i in range(nr):
    t = 0
    p = p0
    while p <= pa:
      p = p * np.exp(m + s * gauss(0,1))
      t += 1
    l.append(t)
  return l

In [ ]:
@jit(nopython = True)
def f4_anti(m, s, p0, pa, nr = 10**6):
  l = []
  for i in range(nr):
    t1, t2 = 0, 0
    p, q = p0, p0
    while p <= pa or q <= pa:
      g = gauss(0,1)
      if p <= pa:
        p = p * np.exp(m + s * g)
        t1 += 1
      elif q <= pa:
        q = q * np.exp(m + s * (-g))
        t2 += 1
    l.append((t1+t2)/2)
  return l

In [ ]:
nr = 10**4
l1 = f4(0.002, 0.01, 5, 10.4, nr)
print(np.mean(l1))
v1 = np.var(l1, ddof = 1)
print(v1/nr)

l2 = f4_anti(0.002, 0.01, 5, 10.4, nr)
print(np.mean(l2))
v2 = np.var(l2, ddof = 1)
print(v2/nr)

print(100 * (v1-v2)/v1)

369.2364
0.9337718086848685
369.31445
0.4714730969294429
49.50874586870754


In [ ]:
nr = 10**6
l = f4(0.002, 0.01, 5, 10.4, nr)
print(np.mean(l))
print(np.var(l, ddof = 1)/nr)

#5

La variable aleatoria X sigue una distribución normal estándar.

A) Estime por simulación $P(X>3)$.

B) Use la técnica de muestreo de importancia para reducir la varianza (use una normal y encuentre los mejores parámetros)

In [ ]:
@jit(nopython = True)
def f5(nr= 10**6):
  l = []
  for i in range(nr):
    r = gauss(0,1)
    if r > 3:
      l.append(1)
    else:
      l.append(0)
  return l

In [ ]:
l = f5()
m = np.mean(l)
v = np.var(l, ddof = 1)
print(m, v)

0.001331 0.0013292297682297683


In [ ]:
@jit(nopython = True)
def f5_impo(m=0,s=1,nr= 10**6, p=3):
  l, k = [], []
  for i in range(nr):
    r = gauss(0,1)
    if r > p:
      I = 1
    else:
      I = 0
    l.append(I)
    g = gauss(m,s)
    f = np.exp(-g**2/2)/np.sqrt(2*np.pi)
    h = np.exp(-(g-m)**2/(2*s**2))/(np.sqrt(2*np.pi)*s)
    if g > p:
      I = 1
    else:
      I = 0
    k.append(I*f/h)
  return l, k

In [ ]:
@jit(nopython = True)
def nunu(r = 10**6):
  may = 0
  mm, ms = 1,1
  for m in np.arange(1,5,0.5):
    for s in np.arange(1,5,0.5):
      l1, l2 = f5_impo(m,s,r)
      l1, l2 = np.array(l1), np.array(l2)
      m1 = l1.mean()
      m2 = l2.mean()
      v1 = l1.var()
      v2 = l2.var()
      if round(100*(v1-v2)/v1, 2) > may:
        may = round(100*(v1-v2)/v1, 2)
        mm, ms = m, s

  return mm, ms, may

r = 10**6
a, b, c = nunu(r)
print(a, b, c)
l1, l2 = f5_impo(a,b,r)
m1 = np.mean(l1)
m2 = np.mean(l2)
v1 = np.var(l1, ddof = 1)
v2 = np.var(l2, ddof = 1)
print(m1, m2)
print(v1, v2)
print(v1/r, v2/r)
print('Reducciòn de varianza:',round(100*(v1-v2)/v1, 2))


3.0 1.0 99.53
0.00137 0.001346293693779444
0.0013681244681244683 6.1510748242830695e-06
1.3681244681244683e-09 6.15107482428307e-12
Reducciòn de varianza: 99.55


#6

$X$ sigue una distribución exponencial con parámetro 1. $Y$ sigue una distribución exponencial con parámetro 2 y $X$ y $Y$ son independientes.

A)Estime per simulación $P(X + Y >5)$

B) Use la técnica de condicionamiento para reducir la varianza, ¿En qué porcentaje se redujo la varianza?

In [ ]:
@jit(nopython = True)
def f6(lam_x=1, lam_y=2, nr=10**6, p = 3):
  l = []
  for i in range(nr):
    x, y = -np.log(random())/lam_x, -np.log(random())/lam_x
    if x + y > p:
      I = 1
    else:
      I = 0
    l.append(I)
  return l

In [ ]:
p = f6()
print(np.mean(p))

0.199011


In [ ]:
@jit(nopython = True)
def f6_cond(lam_x=1, lam_y=2, nr=10**5, p=3):
  l, l1 = [], []
  for i in range(nr):
    x, y = -np.log(random())/lam_x, -np.log(random())/lam_y
    if x + y > p:
      I = 1
    else:
      I = 0
    l.append(I)
    if y > p:
      EIY = 1
    else:
      EIY = np.exp(y-p)
    l1.append(EIY)
  return l, l1

In [ ]:
l1, l2 = f6_cond(p=5)
print(np.mean(l1), np.mean(l2))
v1, v2 = np.var(l1, ddof=1), np.var(l2, ddof=1)
print(v1, v2)
print(100*(v1 - v2)/v1)

0.01343 0.01353576095548946
0.01324976759767597 0.0003386317126278203
97.44424413385774


#7

La variable aleatoria $X$ tiene la siguiente función de densidad de probabilidad

$$f(x) = \left\{ \begin{array} $\frac{x}{2} & \textrm{si } x \in [0,2] \\ 0 & \textrm{en otro caso} \end{array} \right.$$
& $Y$ sigue una distribución exponencial con parámetro 1. $X$ & $Y$ son independientes.

a) Estime por simulación $P(X+Y>2)$.

b) Use la variable aleatoria $E[I|Y]$ para reducir la varianza por condicionamiento. ¿En que pocentaje se redujo la varianza?

c) Use $X+Y$ como variable de control. ¿En qué porcentaje se redujo la varianza?



In [ ]:
@jit(nopython = True)
def dens7(x):
  if 0 <= x <= 2:
    return np.sqrt(4*x)
  else:
    return 0

In [ ]:
@jit(nopython = True)
def f7x(lam_y=1, nr=10**6, p = 2):
  l, l1 = [], []
  for i in range(nr):
    x = dens7(random())
    y = -np.log(random())/lam_y
    if x + y > p:
      I = 1
    else:
      I = 0
    l.append(I)
    if x > p:
      EIY = 1
    else:
      EIY = np.exp(x-p)
    l1.append(EIY)
  return l, l1

@jit(nopython = True)
def f7y(lam_y=1, nr=10**6, p = 2):
  l, l1 = [], []
  for i in range(nr):
    x = dens7(random())
    y = -np.log(random())/lam_y
    if x + y > p:
      I = 1
    else:
      I = 0
    l.append(I)
    if 0 <= y <= p:
      EIY = 1 - ((p-y)**2)/4
    elif y > p:
      EIY = 1
    else:
      EIY = 0
    l1.append(EIY)
  return l, l1

In [ ]:
l1, l2 = f7x(p = 2)
print(np.mean(l1), np.mean(l2))
v1, v2 = np.var(l1, ddof=1), np.var(l2, ddof=1)
print(v1, v2)
print(100*(v1 - v2)/v1)

print()

l1, l2 = f7y(p = 2)
print(np.mean(l1), np.mean(l2))
v1, v2 = np.var(l1, ddof=1), np.var(l2, ddof=1)
print(v1, v2)
print(100*(v1 - v2)/v1)

0.567379 0.5671954954392113
0.24546031581931554 0.05505012989169554
77.5726965444719

0.567537 0.5675726963324951
0.24543899906999908 0.11013635324943302
55.1267917214647


In [ ]:
@jit(nopython = True)
def f7_cont(lam_y=1, nr=10**6, p = 2):
  l, l1, lc = [], [], []
  for i in range(nr):
    x = dens7(random())
    y = -np.log(random())/lam_y
    if x + y > p:
      I = 1
    else:
      I = 0
    l.append(I)
    l1.append(x+y)
  m = np.array(l1).mean()
  c = -np.cov(l,l1)[0,1]/np.cov(l,l1)[1,1]
  for i in range(nr):
    lc.append(l[i] + c*(l1[i]-m))
  return l, lc

In [ ]:
l1, l2 = f7_cont(p = 2)
print(np.mean(l1), np.mean(l2))
v1, v2 = np.var(l1, ddof=1), np.var(l2, ddof=1)
print(v1, v2)
print(100*(v1 - v2)/v1)

0.567447 0.5674469999999894
0.24545114764214754 0.1280513717815444
47.83020042414496
